In [2]:
import pandas as pd
import os
import random
import numpy as np

dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}  

path = '/opt/ml/input/data'

### `feature` 폴더 생성

In [3]:
if not os.path.isdir(os.path.join(path,'feature')):
    os.mkdir(os.path.join(path,'feature'))
os.path.isdir(os.path.join(path,'feature'))

True

### data불러오기
- dtype을 명시해준체로 불러옵니다.
- 불러오면 userID, 시간 순서로 정렬해줍니다.
- test_data는 정답을 맞춰야하는 -1을 제외하고 합쳐줍니다.

In [4]:
train_df = pd.read_csv(os.path.join(path,'train_data.csv'), dtype=dtype, parse_dates=['Timestamp'])
train_df = train_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [5]:
test_df = pd.read_csv(os.path.join(path,'test_data.csv'), dtype=dtype, parse_dates=['Timestamp'])
test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
test_df = test_df[test_df['answerCode']!= -1].copy()

In [6]:
train_df = pd.concat([train_df,test_df])

### feature 추출후 합쳐주기

User, testId 에 따른 feature를 추출하기전에 전체에 공통되는 feature를 추출해줍니다.

- elapsed : 풀는데 걸린 시간
- grade : 대분류
- mid : 중분류
- problem_number : 문제 번호
- correct_t : testId별 정답률
- correct_k : knowledgeTag별 정답률

In [7]:
# testId(시험지)의 평균 정답률을 구합니다.(시헙지별 난이도)
testId_acc = train_df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
testId_acc.columns = ["test_mean", 'test_sum']

# 마찬가지로 KnowledgedTag의 평균 정답률을 구하여 대략적인 난이도를 구합니다
knowLedgedTag_acc = train_df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
knowLedgedTag_acc.columns = ["tag_mean", 'tag_sum']

# 아래 데이터는 제출용 데이터셋에 대해서도 재사용 합니다.
train_df = pd.merge(train_df, testId_acc, on=['testId'], how="left")
train_df = pd.merge(train_df, knowLedgedTag_acc, on=['KnowledgeTag'], how="left")

diff = train_df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
    
# 만약 0초만에 풀었으면 0으로 치환
diff = diff.fillna(pd.Timedelta(seconds=0))

# 시간을 전부 초단위로 변경합니다.
diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

# df에 elapsed(문제 풀이 시간)을 추가해줍니다.
train_df['elapsed'] = diff

# 문제 풀이 시간이 650초 이상은 이상치로 판단하고 제거합니다.
train_df['elapsed'] = train_df['elapsed'].apply(lambda x : x if x <650 else None)

# 대분류(앞 세자리)
train_df['grade'] = train_df['testId'].apply(lambda x : int(x[1:4])//10)

# 중분류(중간 세자리)
train_df['mid'] = train_df['testId'].apply(lambda x : int(x[-3:]))

# 문제 번호(분류를 제외한)
train_df['problem_number'] = train_df['assessmentItemID'].apply(lambda x : int(x[-3:]))


### User의 grade 별로 feature들을 추출합니다
- acc : 대분류 별 정답률
- count	: 해당 대분류 문제를 푼 갯수
- elapsed : 해당 대분류 한문제를 푸는데 걸린 평균 시간

In [8]:
user_feature = train_df.groupby(['userID','grade']).agg({
    'answerCode':['mean', 'count'],
    'elapsed':['mean']
})
user_feature.reset_index(inplace=True)
user_feature

userID grade answerCode           elapsed
                         mean count        mean
0          0     2   0.722222    36   37.206897
1          0     6   0.791908   346   36.533784
2          0     8   0.468320   363   39.261981
3          1     4   0.822719   581   96.679204
4          1     9   0.903409   352  103.672297
...      ...   ...        ...   ...         ...
19734   7439     4   0.733333    15   38.384615
19735   7440     3   0.400000    10   29.125000
19736   7440     5   0.400000     5   18.600000
19737   7441     3   0.200000     5   44.000000
19738   7441     4   1.000000     4   43.333333

[19739 rows x 5 columns]

In [9]:
user_feature.columns = ["userID","grade","grade_acc","grade_count", "grade_elapsed"]


In [10]:
user_feature.head()

,userID,grade,grade_acc,grade_count,grade_elapsed
0,0,2,0.722222,36,37.206897
1,0,6,0.791908,346,36.533784
2,0,8,0.468320,363,39.261981
3,1,4,0.822719,581,96.679204
4,1,9,0.903409,352,103.672297


In [11]:
# 저장해줍니다.
user_feature.to_csv("/opt/ml/input/data/feature/userID_feature.csv")


### testId(시험지) feature 추출
- elapsed : 해당 시험지의 한문제를 푸는데 걸린 평균 시간
- problem_number : 문제 수
- type_number : 유형 수
- grade : 앞 세자리
- mid : 뒤 세자리

In [18]:
len_seq = lambda x : len(set(x))

testId_feature = train_df.groupby(['testId']).agg({
    'elapsed': 'mean',
    'answerCode':['mean', 'sum'],
    'problem_number':'max',
    'KnowledgeTag':len_seq
})
testId_feature.reset_index(inplace=True)
testId_feature['grade']=testId_feature['testId'].apply(lambda x : int(x[1:4])//10)
testId_feature['mid']=testId_feature['testId'].apply(lambda x : int(x[-3:]))

testId_feature.head(5)

testId    elapsed answerCode         problem_number KnowledgeTag grade  \
                    mean       mean     sum            max     <lambda>         
0  A010000001  33.949286   0.923429  1616.0              5            1     1   
1  A010000002  34.218902   0.931429  1630.0              5            2     1   
2  A010000003  36.326211   0.842857  1475.0              5            3     1   
3  A010000004  34.215713   0.880571  1541.0              5            3     1   
4  A010000005  36.883227   0.849143  1486.0              5            2     1   

  mid  
       
0   1  
1   2  
2   3  
3   4  
4   5

In [19]:
testId_feature.columns = ['testId','elapsed','acc','count','problem_number',"type_number","grade","mid"]
testId_feature = testId_feature[["testId","problem_number","type_number","acc","elapsed"]]
testId_feature.columns = [["testId","problem_count","tag_count","testId_acc","testId_elapsed"]]

In [20]:
testId_feature

,testId,problem_count,tag_count,testId_acc,testId_elapsed
0,A010000001,5,1,0.923429,33.949286
1,A010000002,5,2,0.931429,34.218902
2,A010000003,5,3,0.842857,36.326211
3,A010000004,5,3,0.880571,34.215713
4,A010000005,5,2,0.849143,36.883227
...,...,...,...,...,...
1532,A090000070,8,4,0.461153,41.475362
1533,A090000071,8,5,0.345000,40.847059
1534,A090000072,6,2,0.453333,45.295547
1535,A090000073,6,3,0.536667,23.459677


In [21]:
testId_feature.to_csv("/opt/ml/input/data/feature/testId_feature.csv")
